In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import json
import os
from pprintpp import pprint

<h2><b>1. Data uit het applicatieprofiel halen

In [4]:
#haal data uit AP
def getData(link, driver):
    driver.get(link)
    html = driver.page_source
    soup = BeautifulSoup(html, features="lxml")
    
    entiteiten = False
    datatypes = False
    klassen = []
    typedict = []
    
    for d in soup.find_all('div', attrs={"class":"region region--no-space-top"}):
        #entiteiten ophalen
        for h2 in d.find_all('h2'):
            if h2.text == "Entiteiten":
                entiteiten = True
        if entiteiten:
            datatypes = False
            for h3 in d.find_all('h3'): #hier zit je in de box van een klasse
                for a in h3.find_all('a'): #link van de klasse zelf in een dict
                    linkdict = {'link':a['href']}
                for dl in d.find_all('dl'): #hier zit je in de box met de informatie van de klasse zelf
                    beschr = False #steek beschrijving in een dict
                    for dt in dl.find_all(['dt','dd']):
                        if beschr:
                            beschrijving = {'beschrijving':dt.text}
                            beschr = False
                        if dt.text == "Beschrijving":
                            beschr = True
                
                #enkel de beschrijving wordt opgeslagen, de rest kan maar lijkt nutteloos

                #attributen ophalen
                attributen = {}
                for tr in d.find_all('tr', attrs={"typeof":"rdfs:Property"}):
                    lijst = []
                    for td in tr.find_all('td'):
                        geenLink = True
                        for a in td.find_all('a'):
                            attrlink = {}
                            if a.text[5:10] == "     ":
                                attrlink[a.text[29:]] = a['href']
                            else:
                                attrlink[a.text] = a['href']
                            lijst.append(attrlink)
                            geenLink = False
                        if geenLink:
                            lijst.append(td.text)
                    
                    attributen['attributen'] = lijst
                if attributen == {}:
                    attributen['attributen'] = [{'bla':'bla'},{'bla':'bla'},'bla','bla','bla','bla']
                klasse = [linkdict, beschrijving, attributen]
                klassen.append({ h3.text : klasse })
        
        #datatypes ophalen
        for h2 in d.find_all('h2'):
            if h2.text == "Datatypes":
                datatypes = True
        if datatypes:
            entiteiten = False
            for h3 in d.find_all('h3'): #hier zit je in de box van een datatype
                for a in h3.find_all('a'): #link van het datatype zelf in een dict
                    linkdict = {'link':a['href']}
                for dl in d.find_all('dl'): #hier zit je in de box met de informatie van het datatype zelf
                    beschr = False #steek beschrijving in een dict
                    for dt in dl.find_all(['dt','dd']):
                        if beschr:
                            beschrijving = {'beschrijving':dt.text}
                            beschr = False
                        if dt.text == "Beschrijving":
                            beschr = True
                
                #enkel de beschrijving wordt opgeslagen, de rest kan maar lijkt nutteloos

                #attributen ophalen
                attributen = {}
                for tr in d.find_all('tr', attrs={"typeof":"rdfs:Property"}):
                    lijst = []
                    for td in tr.find_all('td'):
                        geenLink = True
                        for a in td.find_all('a'):
                            attrlink = {}
                            if a.text[5:10] == "     ":
                                attrlink[a.text[29:]] = a['href']
                            else:
                                attrlink[a.text] = a['href']
                            lijst.append(attrlink)
                            geenLink = False
                        if geenLink:
                            lijst.append(td.text)
                    
                    attributen['attributen'] = lijst
                if attributen == {}:
                    attributen['attributen'] = [{'bla':'bla'},{'bla':'bla'},'bla','bla','bla','bla']
                datatype = [linkdict, beschrijving, attributen]
                typedict.append({ h3.text : datatype })
        
    
    stanDict = {}
    stanDict["Entiteiten"] = klassen
    stanDict["Datatypes"] = typedict
    return stanDict


<h2><b>2. Data van het applicatieprofiel analyseren

In [5]:
#analyseer het AP
def analyse(data):
    resultaat = ""
    for soort in data:
        for entiteit in data[soort]:
            
            #link van klasse zelf checken
            if "fixme" in list(entiteit.values())[0][0]['link']:
                resultaat += "fixme gevonden in de link van klasse \"{}\"\n".format(list(entiteit.keys())[0])
            
            #beschrijving van de klasse zelf checken
            if list(entiteit.values())[0][1]['beschrijving'] == "":
                resultaat += "Geen beschrijving gevonden bij klasse \"{}\"\n".format(list(entiteit.keys())[0])
            
            #attributen hier overlopen
            attribuut = list(list(list(entiteit.values())[0][2].values())[0][0].keys())[0]
            attributen = list(list(entiteit.values())[0][2].values())[0]
            
            cel = attributen[0]
            if "fixme" in cel[list(cel.keys())[0]]:
                resultaat += "fixme gevonden in attribuut \"{}\" van klasse \"{}\"\n".format(attribuut, list(entiteit.keys())[0])
            cel = attributen[1]
            try:
                if "fixme" in cel[list(cel.keys())[0]]:
                    resultaat += "fixme gevonden in attribuut \"{}\" van klasse \"{}\"\n".format(attribuut, list(entiteit.keys())[0])
            except:
                resultaat += "Geen verwacht type gevonden bij attribuut \"{}\" van klasse \"{}\"\n".format(attribuut, list(entiteit.keys())[0])
            cel = attributen[2]
            if cel == "":
                resultaat += "Kardinaliteit ontbreekt in attribuut \"{}\" van klasse \"{}\"\n".format(attribuut, list(entiteit.keys())[0])
            cel = attributen[3]
            if cel == "":
                resultaat += "Definitie ontbreekt in attribuut \"{}\" van klasse \"{}\"\n".format(attribuut, list(entiteit.keys())[0])
    
    if resultaat == "":
        resultaat += "Alles is in orde!"
    
    return resultaat

<h2><b>3. Conistentie van de database

In [ ]:
def consistentie(database):
    #alle attributen doorlopen
    

<h3><b>3.1 Alle links controleren

In [14]:
driver = webdriver.Chrome()

links = ['https://data.vlaanderen.be/doc/applicatieprofiel/bedrijventerrein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluit-publicatie/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluitvorming', 'https://data.vlaanderen.be/doc/applicatieprofiel/contactvoorkeuren/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-event', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-object', 'https://data.vlaanderen.be/doc/applicatieprofiel/DCAT-AP-VL/erkendestandaard/2021-12-02', 'https://data.vlaanderen.be/doc/applicatieprofiel/DCAT-AP-VL', 'https://data.vlaanderen.be/doc/applicatieprofiel/dienst-transactiemodel/', 'https://data.vlaanderen.be/doc/applicatieprofiel/dossier/', 'https://data.vlaanderen.be/doc/applicatieprofiel/generieke-terugmeldfaciliteit/', 'https://data.vlaanderen.be/doc/applicatieprofiel/issue-tracking-voor-burgers-en-organisaties/', 'https://data.vlaanderen.be/doc/applicatieprofiel/ldes', 'https://data.vlaanderen.be/doc/applicatieprofiel/mandatendatabank/', 'https://data.vlaanderen.be/doc/applicatieprofiel/metadata-dcat/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/dienstregeling-en-planning/stopplaatsen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/dienstregeling-en-planning/tijdstabellen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/dienstregeling-en-planning/voertuigplanning/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit-trips-en-aanbod', 'https://data.vlaanderen.be/doc/applicatieprofiel/notificatie-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/openbaar-domein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/perceel/', 'https://data.vlaanderen.be/doc/applicatieprofiel/logies-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/vervoersknooppunten', 'https://data.vlaanderen.be/doc/applicatieprofiel/vlaamse-codex/', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsenbeheer/', 'https://data.vlaanderen.be/doc/applicatieprofiel/begroeid-voorkomen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/infrastructuurelementen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/onbegroeid-voorkomen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/terreindelen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/vegetatie-elementen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/watervoorkomen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterdelen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsenbeheer', 'https://data.vlaanderen.be/doc/applicatieprofiel/begroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/gebieden', 'https://data.vlaanderen.be/doc/applicatieprofiel/infrastructuurelementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/onbegroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/terreindelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/vegetatie-elementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterdelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/watervoorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/openbaar-domein', 'https://data.vlaanderen.be/doc/applicatieprofiel/openbaar-domein', 'https://data.vlaanderen.be/doc/applicatieprofiel/gebieden', 'https://data.vlaanderen.be/doc/applicatieprofiel/infrastructuurelementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/vegetatie-elementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/terreindelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/onbegroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterdelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/watervoorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsenbeheer', 'https://data.vlaanderen.be/doc/applicatieprofiel/bedrijventerrein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/dossier/', 'https://data.vlaanderen.be/doc/applicatieprofiel/adresregister/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluit-mobiliteit/', 'https://data.vlaanderen.be/doc/applicatieprofiel/subsidieregister/', 'https://data.vlaanderen.be/doc/applicatieprofiel/bestuurlijk-sanctieregister', 'https://data.vlaanderen.be/doc/applicatieprofiel/bodem-en-ondergrond/bodem-en-ondergrond/kandidaatstandaard/2022-04-28', 'https://data.vlaanderen.be/doc/applicatieprofiel/dienstencataloog/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultuurparticipatie', 'https://data.vlaanderen.be/doc/applicatieprofiel/FeitelijkeVerenigingen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/financiele-data', 'https://data.vlaanderen.be/doc/applicatieprofiel/inname-openbaar-domein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/loongegevens', 'https://data.vlaanderen.be/doc/applicatieprofiel/observaties-en-metingen/kandidaatstandaard/2022-04-28', 'https://data.vlaanderen.be/doc/applicatieprofiel/organisatie-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/persoon-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/sensoren-en-bemonstering/kandidaatstandaard/2022-04-28', 'https://data.vlaanderen.be/doc/applicatieprofiel/verkeersborden/', 'https://data.vlaanderen.be/doc/applicatieprofiel/wegenregister/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-object', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-event', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultuur-en-jeugdinfrastructuur', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterkwaliteit', 'https://data.vlaanderen.be/doc/applicatieprofiel/statistiek', 'https://data.vlaanderen.be/doc/applicatieprofiel/datakwaliteit']

database = {}
stdlijst = []
for link in links:
    data = getData(link, driver)
    database[link] = data
    #print(link, "\n")
    #print(resultaat, "\n\n\n")

with open('database.json', 'w', encoding='utf-8') as f:
    json.dump(database, f, ensure_ascii=False, indent=4)


<h3><b>3.2 Een applicatieprofiel controleren

In [7]:
#Haal website op
driver = webdriver.Chrome()
inp = input("Geef de link van het AP: ")
try:
    data = getData(inp, driver)
    resultaat = analyse(data)
    print(resultaat, "\n\n\n")
except:
    print("Werkt niet!")


Alles is in orde! 





<b><h3>3.3 Consistentie van de database

<h1><b>Appendix

<h2>links ophalen vanaf hier</h2>

In [141]:
links = ['https://data.vlaanderen.be/doc/applicatieprofiel/bedrijventerrein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluit-publicatie/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluitvorming', 'https://data.vlaanderen.be/doc/applicatieprofiel/contactvoorkeuren/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-event', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-object', 'https://data.vlaanderen.be/doc/applicatieprofiel/DCAT-AP-VL/erkendestandaard/2021-12-02', 'https://data.vlaanderen.be/doc/applicatieprofiel/DCAT-AP-VL', 'https://data.vlaanderen.be/doc/applicatieprofiel/dienst-transactiemodel/', 'https://data.vlaanderen.be/doc/applicatieprofiel/dossier/', 'https://data.vlaanderen.be/doc/applicatieprofiel/generieke-terugmeldfaciliteit/', 'https://data.vlaanderen.be/doc/applicatieprofiel/issue-tracking-voor-burgers-en-organisaties/', 'https://data.vlaanderen.be/doc/applicatieprofiel/ldes', 'https://data.vlaanderen.be/doc/applicatieprofiel/mandatendatabank/', 'https://data.vlaanderen.be/doc/applicatieprofiel/metadata-dcat/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/dienstregeling-en-planning/stopplaatsen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/dienstregeling-en-planning/tijdstabellen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/dienstregeling-en-planning/voertuigplanning/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit-trips-en-aanbod', 'https://data.vlaanderen.be/doc/applicatieprofiel/notificatie-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/openbaar-domein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/perceel/', 'https://data.vlaanderen.be/doc/applicatieprofiel/logies-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/vervoersknooppunten', 'https://data.vlaanderen.be/doc/applicatieprofiel/vlaamse-codex/', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsenbeheer/', 'https://data.vlaanderen.be/doc/applicatieprofiel/begroeid-voorkomen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/infrastructuurelementen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/onbegroeid-voorkomen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/terreindelen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/vegetatie-elementen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/watervoorkomen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterdelen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsenbeheer', 'https://data.vlaanderen.be/doc/applicatieprofiel/begroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/gebieden', 'https://data.vlaanderen.be/doc/applicatieprofiel/infrastructuurelementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/onbegroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/terreindelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/vegetatie-elementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterdelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/watervoorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/openbaar-domein', 'https://data.vlaanderen.be/doc/applicatieprofiel/openbaar-domein', 'https://data.vlaanderen.be/doc/applicatieprofiel/gebieden', 'https://data.vlaanderen.be/doc/applicatieprofiel/infrastructuurelementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/vegetatie-elementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/terreindelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/onbegroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterdelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/watervoorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsenbeheer', 'https://data.vlaanderen.be/doc/applicatieprofiel/bedrijventerrein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/dossier/', 'https://data.vlaanderen.be/doc/applicatieprofiel/adresregister/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluit-mobiliteit/', 'https://data.vlaanderen.be/doc/applicatieprofiel/subsidieregister/', 'https://data.vlaanderen.be/doc/applicatieprofiel/bestuurlijk-sanctieregister', 'https://data.vlaanderen.be/doc/applicatieprofiel/bodem-en-ondergrond/bodem-en-ondergrond/kandidaatstandaard/2022-04-28', 'https://data.vlaanderen.be/doc/applicatieprofiel/dienstencataloog/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultuurparticipatie', 'https://data.vlaanderen.be/doc/applicatieprofiel/FeitelijkeVerenigingen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/financiele-data', 'https://data.vlaanderen.be/doc/applicatieprofiel/inname-openbaar-domein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/loongegevens', 'https://data.vlaanderen.be/doc/applicatieprofiel/observaties-en-metingen/kandidaatstandaard/2022-04-28', 'https://data.vlaanderen.be/doc/applicatieprofiel/organisatie-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/persoon-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/sensoren-en-bemonstering/kandidaatstandaard/2022-04-28', 'https://data.vlaanderen.be/doc/applicatieprofiel/verkeersborden/', 'https://data.vlaanderen.be/doc/applicatieprofiel/wegenregister/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-object', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-event', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultuur-en-jeugdinfrastructuur', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterkwaliteit', 'https://data.vlaanderen.be/doc/applicatieprofiel/statistiek', 'https://data.vlaanderen.be/doc/applicatieprofiel/datakwaliteit']

In [139]:
def getLinks(driver):
    driver.get('https://data.vlaanderen.be/standaarden/')
    html = driver.page_source
    soup = BeautifulSoup(html, features="lxml")

    dict = {}
    for tag in soup.find_all('a', href=True):
        titel = tag.text

        if "erkende-standaard" in tag['href'] or "kandidaat-standaard" in tag['href']:
            if tag['href'][:4] == "http":
                link = tag['href']
            else:
                link = "https://data.vlaanderen.be" + tag['href']
            
            dict[titel] = link
    
    return dict


def getStandaard(link, driver):
    driver.get(link)
    html = driver.page_source
    soup = BeautifulSoup(html, features="lxml")

    lijst = []
    for art in soup.find_all('article'):
        specdoc = False
        
        for h1 in art.find_all('h1'):
            if h1.text == "Specificatiedocument":
                specdoc = True
        
        if specdoc == True:
            for a in art.find_all('a'):
                lijst.append(a['href'])
        
        return lijst

def update():
    driver = webdriver.Chrome()
    links = getLinks(driver)
    
    linklijst = []
    for link in links.values():
        for l in getStandaard(link, driver):
            linklijst.append(l)
    
    #resultaat = []
    #for a in linklijst:
    #    if 'applicatieprofiel' in a:
    #        resultaat.append(a)
    print(resultaat)

update()
    


['https://data.vlaanderen.be/doc/applicatieprofiel/bedrijventerrein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluit-publicatie/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluitvorming', 'https://data.vlaanderen.be/doc/applicatieprofiel/contactvoorkeuren/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-event', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-object', 'https://data.vlaanderen.be/doc/applicatieprofiel/DCAT-AP-VL/erkendestandaard/2021-12-02', 'https://data.vlaanderen.be/doc/applicatieprofiel/DCAT-AP-VL', 'https://data.vlaanderen.be/doc/applicatieprofiel/dienst-transactiemodel/', 'https://data.vlaanderen.be/doc/applicatieprofiel/dossier/', 'https://data.vlaanderen.be/doc/applicatieprofiel/generieke-terugmeldfaciliteit/', 'https://data.vlaanderen.be/doc/applicatieprofiel/issue-tracking-voor-burgers-en-organisaties/', 'https://data.vlaanderen.be/doc/applicatieprofiel/ldes', 'https://data.vlaanderen.be/doc/applic

In [140]:
len(['https://data.vlaanderen.be/doc/applicatieprofiel/bedrijventerrein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluit-publicatie/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluitvorming', 'https://data.vlaanderen.be/doc/applicatieprofiel/contactvoorkeuren/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-event', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-object', 'https://data.vlaanderen.be/doc/applicatieprofiel/DCAT-AP-VL/erkendestandaard/2021-12-02', 'https://data.vlaanderen.be/doc/applicatieprofiel/DCAT-AP-VL', 'https://data.vlaanderen.be/doc/applicatieprofiel/dienst-transactiemodel/', 'https://data.vlaanderen.be/doc/applicatieprofiel/dossier/', 'https://data.vlaanderen.be/doc/applicatieprofiel/generieke-terugmeldfaciliteit/', 'https://data.vlaanderen.be/doc/applicatieprofiel/issue-tracking-voor-burgers-en-organisaties/', 'https://data.vlaanderen.be/doc/applicatieprofiel/ldes', 'https://data.vlaanderen.be/doc/applicatieprofiel/mandatendatabank/', 'https://data.vlaanderen.be/doc/applicatieprofiel/metadata-dcat/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/dienstregeling-en-planning/stopplaatsen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/dienstregeling-en-planning/tijdstabellen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/dienstregeling-en-planning/voertuigplanning/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit-trips-en-aanbod', 'https://data.vlaanderen.be/doc/applicatieprofiel/notificatie-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/openbaar-domein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/perceel/', 'https://data.vlaanderen.be/doc/applicatieprofiel/logies-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/mobiliteit/vervoersknooppunten', 'https://data.vlaanderen.be/doc/applicatieprofiel/vlaamse-codex/', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsenbeheer/', 'https://data.vlaanderen.be/doc/applicatieprofiel/begroeid-voorkomen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/infrastructuurelementen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/onbegroeid-voorkomen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/terreindelen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/vegetatie-elementen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/watervoorkomen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterdelen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsenbeheer', 'https://data.vlaanderen.be/doc/applicatieprofiel/begroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/gebieden', 'https://data.vlaanderen.be/doc/applicatieprofiel/infrastructuurelementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/onbegroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/terreindelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/vegetatie-elementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterdelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/watervoorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/openbaar-domein', 'https://data.vlaanderen.be/doc/applicatieprofiel/openbaar-domein', 'https://data.vlaanderen.be/doc/applicatieprofiel/gebieden', 'https://data.vlaanderen.be/doc/applicatieprofiel/infrastructuurelementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/vegetatie-elementen', 'https://data.vlaanderen.be/doc/applicatieprofiel/terreindelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/onbegroeid-voorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterdelen', 'https://data.vlaanderen.be/doc/applicatieprofiel/watervoorkomen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsen', 'https://data.vlaanderen.be/doc/applicatieprofiel/begraafplaatsenbeheer', 'https://data.vlaanderen.be/doc/applicatieprofiel/bedrijventerrein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/dossier/', 'https://data.vlaanderen.be/doc/applicatieprofiel/adresregister/', 'https://data.vlaanderen.be/doc/applicatieprofiel/besluit-mobiliteit/', 'https://data.vlaanderen.be/doc/applicatieprofiel/subsidieregister/', 'https://data.vlaanderen.be/doc/applicatieprofiel/bestuurlijk-sanctieregister', 'https://data.vlaanderen.be/doc/applicatieprofiel/bodem-en-ondergrond/bodem-en-ondergrond/kandidaatstandaard/2022-04-28', 'https://data.vlaanderen.be/doc/applicatieprofiel/dienstencataloog/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultuurparticipatie', 'https://data.vlaanderen.be/doc/applicatieprofiel/FeitelijkeVerenigingen/', 'https://data.vlaanderen.be/doc/applicatieprofiel/financiele-data', 'https://data.vlaanderen.be/doc/applicatieprofiel/inname-openbaar-domein/', 'https://data.vlaanderen.be/doc/applicatieprofiel/loongegevens', 'https://data.vlaanderen.be/doc/applicatieprofiel/observaties-en-metingen/kandidaatstandaard/2022-04-28', 'https://data.vlaanderen.be/doc/applicatieprofiel/organisatie-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/persoon-basis/', 'https://data.vlaanderen.be/doc/applicatieprofiel/sensoren-en-bemonstering/kandidaatstandaard/2022-04-28', 'https://data.vlaanderen.be/doc/applicatieprofiel/verkeersborden/', 'https://data.vlaanderen.be/doc/applicatieprofiel/wegenregister/', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-object', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultureel-erfgoed-event', 'https://data.vlaanderen.be/doc/applicatieprofiel/cultuur-en-jeugdinfrastructuur', 'https://data.vlaanderen.be/doc/applicatieprofiel/waterkwaliteit', 'https://data.vlaanderen.be/doc/applicatieprofiel/statistiek', 'https://data.vlaanderen.be/doc/applicatieprofiel/datakwaliteit'])

81